# Launch a TigerGraph Developer instance on AWS 
Use the following command in a shell to connect to it 

`ssh -L14240:localhost:14240 -L 8888:localhost:8888 -i your_aws_key.pem ubuntu@your-server-ip`

Inside the server clone the autoTigerGraph repository

`git clone https://github.com/yaniv256/autoTigerGraph`

Run `jupyter notebook` on the server and open this notebook from the `notebooks` directory 

Make sure that the GSQL server is running with 

In [1]:
!gadmin start all

[   Info] Starting EXE
[   Info] Starting CTRL
[   Info] Starting ZK ETCD DICT KAFKA ADMIN GSE NGINX GPE RESTPP KAFKASTRM-LL KAFKACONN TS3SERV GSQL TS3 IFM GUI


Check the connection by clicking on the link below. It should open TigerStudio in a browser window

In [2]:
server = 'http://localhost'
print(server+':14240')

http://localhost:14240


## Setup working enviroment 

In [3]:
!pip install kaggle ijson pyTigerGraph

## Install autoTigerGraph from local respository
You'll need to restart the kernal after this one (keyboard shortcut 0,0):

In [4]:
!pip install -e ..

Obtaining file:///home/ubuntu/autoTigerGraph
  Attempting uninstall: autoTigerGraph
    Found existing installation: autoTigerGraph 0.0.1
    Uninstalling autoTigerGraph-0.0.1:
      Successfully uninstalled autoTigerGraph-0.0.1
  Running setup.py develop for autoTigerGraph


## Load Yelp Dataset from Kaggle

see https://www.kaggle.com/docs/api on how to obtain a kaggle token. Run the next cell and upload your token. 

In [5]:
from ipywidgets import FileUpload
upload = FileUpload()
upload

FileUpload(value={}, description='Upload')

In [6]:
import os

!mkdir -p ~/.kaggle
with open(os.path.expanduser("~/.kaggle/kaggle.json"), "w+b") as i:
    i.write(upload.data[0])
    
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download yelp-dataset/yelp-dataset

100%|█████████████████████████████████████▉| 4.48G/4.48G [01:30<00:00, 82.9MB/s]
100%|██████████████████████████████████████| 4.48G/4.48G [01:30<00:00, 53.4MB/s]


In [11]:
!unzip -o yelp-dataset

Archive:  yelp-dataset.zip
  inflating: Dataset_Agreement.pdf   
  inflating: yelp_academic_dataset_business.json  
  inflating: yelp_academic_dataset_checkin.json  
  inflating: yelp_academic_dataset_review.json  
  inflating: yelp_academic_dataset_tip.json  
  inflating: yelp_academic_dataset_user.json  


In [12]:
ls -l

total 14911464
-rw-rw-r-- 1 ubuntu ubuntu      41776 Mar 26 01:18 Dataset_Agreement.pdf
-rw-rw-r-- 1 ubuntu ubuntu      35460 Jul  2 19:44 YelpGraph.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 4809540040 Jul  2 19:41 yelp-dataset.zip
-rw-rw-r-- 1 ubuntu ubuntu  152898689 Mar 26 01:18 yelp_academic_dataset_business.json
-rw-rw-r-- 1 ubuntu ubuntu  449663480 Mar 26 01:18 yelp_academic_dataset_checkin.json
-rw-rw-r-- 1 ubuntu ubuntu 6325565224 Mar 26 01:19 yelp_academic_dataset_review.json
-rw-rw-r-- 1 ubuntu ubuntu  263489322 Mar 26 01:31 yelp_academic_dataset_tip.json
-rw-rw-r-- 1 ubuntu ubuntu 3268069927 Mar 26 01:32 yelp_academic_dataset_user.json


In [13]:
!rm yelp-dataset.zip

In [45]:
!/bin/bash -c 'for i in *.json; do echo $i; head -1 $i; echo; done'

yelp_academic_dataset_business.json
{"business_id":"f9NumwFMBDn751xgFiRbNA","name":"The Range At Lake Norman","address":"10913 Bailey Rd","city":"Cornelius","state":"NC","postal_code":"28031","latitude":35.4627242,"longitude":-80.8526119,"stars":3.5,"review_count":36,"is_open":1,"attributes":{"BusinessAcceptsCreditCards":"True","BikeParking":"True","GoodForKids":"False","BusinessParking":"{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}","ByAppointmentOnly":"False","RestaurantsPriceRange2":"3"},"categories":"Active Life, Gun\/Rifle Ranges, Guns & Ammo, Shopping","hours":{"Monday":"10:0-18:0","Tuesday":"11:0-20:0","Wednesday":"10:0-18:0","Thursday":"11:0-20:0","Friday":"11:0-20:0","Saturday":"11:0-20:0","Sunday":"13:0-18:0"}}

yelp_academic_dataset_checkin.json
{"business_id":"--1UhMGODdWsrMastO9DZw","date":"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016-10-15 02:45:18, 2016-11-18 01:54:50, 2017-04-20 18:39:06, 2017-05-03 17:58:02, 2019-03-19 22:04:48"

## Make changes if you want to autoTigerGraph.py

In [68]:
%%writefile ../autoTigerGraph/autoTigerGraph.py
import ijson

print('Loaded local dev copy of autoTigerGraph!')

def get_first(filename):

    with open(filename, 'r') as f:
        objects = ijson.items(f, '', multiple_values=True, use_float=True)
        first  = objects.__next__()
        fields = list(zip(range(len(first)), 
                        list(first.keys()), 
                        [type(value).__name__ for value in first.values()]))

    return first, fields


def vertex_from_json(json_object, vertex_name, fields=None):

    if fields == None:
        fields = json_object.keys()
    
    type_translate = {'str': 'STRING', 'dict': 'STRING', 'float': 'DOUBLE', 
                      'bool': 'BOOL', 'int': 'INT'}

    gsql_cmd  = 'CREATE VERTEX ' + vertex_name + ' (PRIMARY_ID ' 
    
    for field in fields:

        if field == 'date':
            gsql_cmd += 'date_time' + ' '
            gsql_cmd += 'DATETIME'
        else:    
            gsql_cmd += field + ' '
            gsql_cmd += type_translate[type(json_object[field]).__name__]
    
        gsql_cmd += ', '

    return gsql_cmd[:-2] + ')'

def problem_fields_to_str(json_object):

    for key, value in json_object.items():
        if isinstance(value, dict) or value == None:
            json_object[key] = str(value)
        if key == 'date':
            json_object.pop(key)
            json_object['date_time'] = value

    return json_object


def upsert_json_vertices(filename, conn, vertex_name, primary_id, batch_size, max_verts):

    ids = ['']*batch_size
    bodies = ['']*batch_size

    with open(filename, 'r') as f:
        objects = ijson.items(f, '', multiple_values=True, use_float=True)

        i = 0
        count = 0
        for json_object in objects:

            if json_object:
                ids[i]=json_object.pop(primary_id)
                bodies[i]=problem_fields_to_str(json_object)
                count += 1
                i += 1
                if count >= max_verts:
                    break
                if i%batch_size == 0:
                    conn.upsertVertices(vertex_name, list(zip(ids, bodies)))
                    i = 0

            else:
                break
                
    conn.upsertVertices(vertex_name, list(zip(ids[:i], bodies[:i])))

    return count

def upsert_json_edges(filename, conn, from_name, from_field, edge_name, 
                      to_name, to_field, batch_size, max_edges, split_string=', '):

    froms = ['']*batch_size
    tos = ['']*batch_size

    with open(filename, 'r') as f:
        objects = ijson.items(f, '', multiple_values=True, use_float=True)

        i = 0
        count = 0
        for json_object in objects:

            if json_object:
                
                json_froms = json_object[from_field].split(split_string)
                jsom_tos = json_object[to_field].split(split_string)
                
                if len(json_from) > 1:
                    json_to *= len(json_form)
                elif len(json_to) > 1:
                    json_from *= len(json_to)
          
                for froms[i], tos[i] in zip(json_from, json_to):

                    count += 1
                    i += 1
                    if count >= max_edges:
                        break
                    if i%n == 0:
                        conn.upsertEdges(from_name, edge_name, to_name, list(zip(froms, tos)))
                        i = 0

            else:
                break

    return count

Overwriting ../autoTigerGraph/autoTigerGraph.py


You'll need to restart the kernal before this one (keyboard shortcut 0,0):

In [1]:
import autoTigerGraph as atg

Loaded local dev copy of autoTigerGraph!


In [2]:
server = 'http://localhost'
print(server+':14240')

http://localhost:14240


In [3]:
import pyTigerGraph as tg

conn = tg.TigerGraphConnection(host=server, graphname='yelp')
shell = tg.Gsql(conn , certNeeded=False)

shell.jarLocation='/home/ubuntu/tigergraph/app/3.0.0/dev/gdk/gsql/lib'

In [4]:
print(shell.gsql('ls', options=[]))

Connecting to localhost:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
---- Global vertices, edges, and all graphs
Vertex Types: 
  - VERTEX Business(PRIMARY_ID business_id STRING, name STRING, address STRING, city STRING, state STRING, postal_code STRING, latitude DOUBLE, longitude DOUBLE, stars DOUBLE, review_count INT, is_open INT, attributes STRING, categories STRING, hours STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
Edge Types: 

Graphs: 
  - Graph yelp(Business:v)
Jobs: 


JSON API version: v2
Syntax version: v1




In [27]:
print(shell.gsql('drop all', options=[]))

Connecting to localhost:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
Dropping all, about 1 minute ...
Abort all active loading jobs
Try to abort all loading jobs on graph yelp, it may take a while ...
[ABORT_SUCCESS] No active Loading Job to abort.
Resetting GPE...
Successfully reset GPE
Stopping GPE GSE
Successfully stopped GPE GSE in 8.037 seconds
Clearing graph store...
Successfully cleared graph store
Everything is dropped.



In [28]:
print(shell.gsql('ls', options=[]))

Connecting to localhost:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
---- Global vertices, edges, and all graphs
Vertex Types: 
Edge Types: 

Graphs: 
Jobs: 


JSON API version: v2
Syntax version: v1




# Create schema and load data

In [29]:
print(shell.gsql('create graph yelp (*)', options=[]))

Connecting to localhost:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
Stopping GPE GSE RESTPP
Successfully stopped GPE GSE RESTPP in 0.005 seconds
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.076 seconds
The graph yelp is created.



## Create and upsert Business

In [42]:
filename = 'yelp_academic_dataset_business.json'
vertex_name = 'Business'
primary_id = 'business_id'

json_object, fields = atg.get_first(filename)
json_object, fields

({'business_id': 'f9NumwFMBDn751xgFiRbNA',
  'name': 'The Range At Lake Norman',
  'address': '10913 Bailey Rd',
  'city': 'Cornelius',
  'state': 'NC',
  'postal_code': '28031',
  'latitude': 35.4627242,
  'longitude': -80.8526119,
  'stars': 3.5,
  'review_count': 36,
  'is_open': 1,
  'attributes': {'BusinessAcceptsCreditCards': 'True',
   'BikeParking': 'True',
   'GoodForKids': 'False',
   'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
   'ByAppointmentOnly': 'False',
   'RestaurantsPriceRange2': '3'},
  'categories': 'Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping',
  'hours': {'Monday': '10:0-18:0',
   'Tuesday': '11:0-20:0',
   'Wednesday': '10:0-18:0',
   'Thursday': '11:0-20:0',
   'Friday': '11:0-20:0',
   'Saturday': '11:0-20:0',
   'Sunday': '13:0-18:0'}},
 [(0, 'business_id', 'str'),
  (1, 'name', 'str'),
  (2, 'address', 'str'),
  (3, 'city', 'str'),
  (4, 'state', 'str'),
  (5, 'postal_code', 'str'),
  (6,

In [43]:
create = atg.vertex_from_json(json_object=json_object, vertex_name=vertex_name)
create

'CREATE VERTEX Business (PRIMARY_ID business_id STRING, name STRING, address STRING, city STRING, state STRING, postal_code STRING, latitude DOUBLE, longitude DOUBLE, stars DOUBLE, review_count INT, is_open INT, attributes STRING, categories STRING, hours STRING)'

In [44]:
print(shell.gsql(
'''
drop graph yelp
drop vertex {}
{}
create graph yelp (*)
ls
'''.format(vertex_name, create)))

Connecting to localhost:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
The graph yelp is dropped.
The vertex type Business is dropped.
The vertex type Business is created.
The graph yelp is created.
---- Graph yelp
Vertex Types: 
  - VERTEX User(PRIMARY_ID user_id STRING, name STRING, review_count INT, yelping_since STRING, useful INT, funny INT, cool INT, elite STRING, friends STRING, fans INT, average_stars DOUBLE, compliment_hot INT, compliment_more INT, compliment_profile INT, compliment_cute INT, compliment_list INT, compliment_note INT, compliment_plain INT, compliment_cool INT, compliment_funny INT, compliment_writer INT, compliment_photos INT) WITH STATS="OUTDEGREE_BY_EDGETYPE"
  - VERTEX Business(PRIMARY_ID business_id STRING, name STRING, address STRING, city STRING, state STRING, postal_code STRING, latitude DOUBLE, longitude DOUBLE, stars DOUBLE, review_count INT, is_open INT, attributes STRING, categories STRING, hours STRING) WITH STA

In [45]:
atg.upsert_json_vertices(filename=filename, conn=conn, vertex_name=vertex_name, 
                      primary_id=primary_id, batch_size=10000, max_verts = 10)

10

In [46]:
shell.gsql('select count(*) from {}'.format(vertex_name))

[{'count': 10, 'v_type': 'Business'}]

In [47]:
atg.upsert_json_vertices(filename=filename, conn=conn, vertex_name=vertex_name, 
                      primary_id=primary_id, batch_size=10000, max_verts = 10**10)

209393

In [48]:
shell.gsql('select count(*) from {}'.format(vertex_name))

[{'count': 209393, 'v_type': 'Business'}]

## Create and upsert User

In [49]:
filename = 'yelp_academic_dataset_user.json'
vertex_name = 'User'
primary_id = 'user_id'

json_object, fields = atg.get_first(filename)
json_object, fields

({'user_id': 'ntlvfPzc8eglqvk92iDIAw',
  'name': 'Rafael',
  'review_count': 553,
  'yelping_since': '2007-07-06 03:27:11',
  'useful': 628,
  'funny': 225,
  'cool': 227,
  'elite': '',
  'friends': 'oeMvJh94PiGQnx_6GlndPQ, wm1z1PaJKvHgSDRKfwhfDg, IkRib6Xs91PPW7pon7VVig, A8Aq8f0-XvLBcyMk2GJdJQ, eEZM1kogR7eL4GOBZyPvBA, e1o1LN7ez5ckCpQeAab4iw, _HrJVzFaRFUhPva8cwBjpQ, pZeGZGzX-ROT_D5lam5uNg, 0S6EI51ej5J7dgYz3-O0lA, woDt8raW-AorxQM_tIE2eA, hWUnSE5gKXNe7bDc8uAG9A, c_3LDSO2RHwZ94_Q6j_O7w, -uv1wDiaplY6eXXS0VwQiA, QFjqxXn3acDC7hckFGUKMg, ErOqapICmHPTN8YobZIcfQ, mJLRvqLOKhqEdkgt9iEaCQ, VKX7jlScJSA-ja5hYRw12Q, ijIC9w5PRcj3dWVlanjZeg, CIZGlEw-Bp0rmkP8M6yQ9Q, OC6fT5WZ8EU7tEVJ3bzPBQ, UZSDGTDpycDzrlfUlyw2dQ, deL6e_z9xqZTIODKqnvRXQ, 5mG2ENw2PylIWElqHSMGqg, Uh5Kug2fvDd51RYmsNZkGg, 4dI4uoShugD9z84fYupelQ, EQpFHqGT9Tk6YSwORTtwpg, o4EGL2-ICGmRJzJ3GxB-vw, s8gK7sdVzJcYKcPv2dkZXw, vOYVZgb_GVe-kdtjQwSUHw, wBbjgHsrKr7BsPBrQwJf2w, p59u2EC_qcmCmLeX1jCi5Q, VSAZI1eHDrOPRWMK4Q2DIQ, efMfeI_dkhpeGykaRJqxfQ, x6qYcQ8

In [50]:
create = atg.vertex_from_json(json_object=json_object, vertex_name=vertex_name)
create

'CREATE VERTEX User (PRIMARY_ID user_id STRING, name STRING, review_count INT, yelping_since STRING, useful INT, funny INT, cool INT, elite STRING, friends STRING, fans INT, average_stars DOUBLE, compliment_hot INT, compliment_more INT, compliment_profile INT, compliment_cute INT, compliment_list INT, compliment_note INT, compliment_plain INT, compliment_cool INT, compliment_funny INT, compliment_writer INT, compliment_photos INT)'

In [51]:
print(shell.gsql(
'''
drop graph yelp
drop vertex {}
{}
create graph yelp (*)
ls
'''.format(vertex_name, create)))

Connecting to localhost:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
The graph yelp is dropped.
The vertex type User is dropped.
The vertex type User is created.
The graph yelp is created.
---- Graph yelp
Vertex Types: 
  - VERTEX Business(PRIMARY_ID business_id STRING, name STRING, address STRING, city STRING, state STRING, postal_code STRING, latitude DOUBLE, longitude DOUBLE, stars DOUBLE, review_count INT, is_open INT, attributes STRING, categories STRING, hours STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
  - VERTEX User(PRIMARY_ID user_id STRING, name STRING, review_count INT, yelping_since STRING, useful INT, funny INT, cool INT, elite STRING, friends STRING, fans INT, average_stars DOUBLE, compliment_hot INT, compliment_more INT, compliment_profile INT, compliment_cute INT, compliment_list INT, compliment_note INT, compliment_plain INT, compliment_cool INT, compliment_funny INT, compliment_writer INT, compliment_photos INT) WITH STATS="OUTD

In [52]:
atg.upsert_json_vertices(filename=filename, conn=conn, vertex_name=vertex_name, 
                      primary_id=primary_id, batch_size=10000, max_verts = 10)

10

In [54]:
shell.gsql('select count(*) from {}'.format(vertex_name))

[{'count': 10, 'v_type': 'User'}]

In [55]:
atg.upsert_json_vertices(filename=filename, conn=conn, vertex_name=vertex_name, 
                      primary_id=primary_id, batch_size=10000, max_verts = 10**10)

1968703

In [56]:
shell.gsql('select count(*) from {}'.format(vertex_name))

[{'count': 1968703, 'v_type': 'User'}]